# Dependencies

In [36]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import re
import sys
from datetime import datetime, date
from datetime import timedelta
import pandas as pd

# Get the links of all news articles within a specific range

In [37]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [38]:
dates = []
# Select the start and end date to search on archive
start_date = date(2024,6,1)
end_date = date(2024,6,30)

for single_Date in daterange(start_date, end_date):
    dates.append(single_Date.strftime("%Y-%m-%d"))

options = Options()
options.add_argument("window-size=1920,1080")
service = Service(executable_path="/Users/ashrafulshuvon/Desktop/Selenium/chromedriver")
driver = webdriver.Chrome(service=service, options=options)
links = []

for date in dates:
    url = 'https://www.ittefaq.com.bd/archive/' + str(date)
    driver.get(url)
    time.sleep(5)
    while True:
        try:
            # If the 'show more' button is not visible, then break the loop & extract the links
            mn_button = driver.find_element(By.CSS_SELECTOR, "#ajax_load_more_565_btn")
            if mn_button.get_attribute('style') == "visibility: hidden;":
                print("End of news")
                time.sleep(5)
                link_elements = driver.find_elements(By.CSS_SELECTOR, "a.link_overlay")
                for element in link_elements:
                    links.append(element.get_attribute('href'))
                print("Links extracted")    
                break
            
            else:
                # Click the 'show more' button untill it is not visible/no more newsleft at that range
                more_button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "#ajax_load_more_565_btn"))
                )

                driver.execute_script("arguments[0].scrollIntoView()", more_button)
                driver.execute_script("arguments[0].click()", more_button)
                print("Clicked")
                time.sleep(5)
            #auto_load_div = driver.find_element(By.CSS_SELECTOR, "#ajax_load_more_565_btn")
            #if auto_load_div.get_attribute('style') == "style: hidden":
            #   print("Visible")

        except ElementClickInterceptedException:
            print("Why?")
driver.quit()

Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clicked
Clicked
Clicked
Clicked
End of news
Links extracted
Clic

In [39]:
lf = pd.DataFrame(links, columns=['links'])

In [40]:
lf.head(10)

,links
0,https://www.ittefaq.com.bd/689262/%E0%A6%B0%E0...
1,https://www.ittefaq.com.bd/689261/%E0%A6%A8%E0...
2,https://www.ittefaq.com.bd/689260/%E0%A6%86%E0...
3,https://www.ittefaq.com.bd/689254/%E0%A6%B0%E0...
4,https://www.ittefaq.com.bd/689202/%E0%A6%A6%E0...
5,https://www.ittefaq.com.bd/689257/%E0%A6%A8%E0...
6,https://www.ittefaq.com.bd/689259/%E0%A6%AA%E0...
7,https://www.ittefaq.com.bd/689258/%E0%A6%A6%E0...
8,https://www.ittefaq.com.bd/689256/%E0%A6%9F%E0...
9,https://www.ittefaq.com.bd/689255/%E0%A6%B2%E0...


In [41]:
lf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2743 entries, 0 to 2742
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   links   2743 non-null   object
dtypes: object(1)
memory usage: 21.6+ KB


In [42]:
lf.to_csv(f'(Links From {start_date}) TO ({end_date}).csv', index=False)

# Now using the links stored perviously, go to each links and extract the news article, summary, headline

There may be some exception errors because of not finding the elements. That means that, certain article page either has only pictures/videos or the page structure is different than the other ones. In that case article, summararies and headlines for those pages nothin will be saved. To handle those cells with missing values, a code is provided.

In [43]:
service = Service(executable_path="/Users/ashrafulshuvon/Desktop/Selenium/chromedriver")
options = Options()
options.add_argument("window-size=1920,1080")
options.page_load_strategy = 'none'
options.add_argument("--headless")  # Enable headless mode
options.add_argument("--blink-settings=imagesEnabled=false")  # Disable images
options.add_argument("--disable-extensions")  # Disable extensions
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(15)

cf = pd.read_csv("(Links From 2024-06-01) TO (2024-06-30).csv")
cnt = 1
data = []
for url in cf['links']:
    try:
        driver.get(url)
        # Wait until the content area is loaded
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.content_detail_left_inner")))  
    except TimeoutException:
        print(f"Timeout while loading {url}")
        continue  # Skip this URL and continue with the next one

    # Initialize variables to hold content
    summary = ""
    headline = ""
    article = []
    
    # Extract the headline text from the <h1> tag
    try:
        headline_element = driver.find_element(By.CSS_SELECTOR, "div.content_detail_left_inner h1")
        headline = headline_element.text
    except Exception as e:
        print("Error retrieving headline:", e)
        
    # Extract the summary text from the first <p> tag(Taking the first paragraph of the article as summary)
    try:
        summary_element = driver.find_element(By.CSS_SELECTOR, "div.content_detail_content_inner p")
        summary = summary_element.text
    except Exception as e:
        print("Error retrieving summary. Use the headline as summary instead", e)
        summary = headline

    # Extract all paragraph texts within the same div
    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "div.content_detail_content_inner p")
        for paragraph in paragraphs:
            # Add text of each paragraph to the article list, ensure it does not contain newlines
            article.append(paragraph.text.replace('\n', ' '))
    except Exception as e:
        print("Error retrieving article paragraphs:", e)

    # Concatenate all article texts into one string separated by spaces
    article_text = ' '.join(article)

    # Create a DataFrame
    data.append({
        'headline': headline,
        'summary': summary,
        'article': article_text
    })
    print(f"Article {cnt} extracted")
    cnt += 1
    
df = pd.DataFrame(data)
time.sleep(5)
# Print and save the DataFrame


# Close the driver
driver.quit()

Article 1 extracted
Article 2 extracted
Article 3 extracted
Article 4 extracted
Article 5 extracted
Article 6 extracted
Article 7 extracted
Article 8 extracted
Article 9 extracted
Article 10 extracted
Article 11 extracted
Article 12 extracted
Article 13 extracted
Article 14 extracted
Article 15 extracted
Article 16 extracted
Article 17 extracted
Article 18 extracted
Article 19 extracted
Article 20 extracted
Article 21 extracted
Article 22 extracted
Article 23 extracted
Article 24 extracted
Article 25 extracted
Article 26 extracted
Article 27 extracted
Article 28 extracted
Article 29 extracted
Article 30 extracted
Article 31 extracted
Article 32 extracted
Article 33 extracted
Article 34 extracted
Article 35 extracted
Article 36 extracted
Article 37 extracted
Article 38 extracted
Article 39 extracted
Article 40 extracted
Article 41 extracted
Article 42 extracted
Article 43 extracted
Article 44 extracted
Article 45 extracted
Article 46 extracted
Article 47 extracted
Article 48 extracted
A

In [44]:
df 


,headline,summary,article
0,রোহিঙ্গা সংকট সমাধানে সমন্বিত প্রচেষ্টা নিন: জ...,দেশে আশ্রিত প্রায় ১৩ লাখ রোহিঙ্গার নিজ দেশ মিয়...,দেশে আশ্রিত প্রায় ১৩ লাখ রোহিঙ্গার নিজ দেশ মিয়...
1,নোয়াব সদস্য সম্মাননা পেলো দৈনিক ইত্তেফাক,প্রকাশনার ৫০ বছর পূর্ণ করায় সংবাদপত্র মালিকদের...,প্রকাশনার ৫০ বছর পূর্ণ করায় সংবাদপত্র মালিকদের...
2,আওয়ামী লীগের ‘প্লাটিনাম জয়ন্তী’ উপলক্ষে ১০ দফা...,২৩ জুন দেশের প্রাচীন ও বৃহত্তম রাজনৈতিক সংগঠন ...,২৩ জুন দেশের প্রাচীন ও বৃহত্তম রাজনৈতিক সংগঠন ...
3,রাজধানীতে মাদকবিরোধী অভিযানে গ্রেপ্তার ১৯,রাজধানীর বিভিন্ন এলাকায় অভিযান চালিয়ে মাদক বিক...,রাজধানীর বিভিন্ন এলাকায় অভিযান চালিয়ে মাদক বিক...
4,দাম্পত্য কলহে সন্তানের মানসিক স্বাস্থ্যে প্রভাব,দাম্পত্য জীবনে ঝগড়া হতেই পারে। তবে এই ঝগড়ার নে...,দাম্পত্য জীবনে ঝগড়া হতেই পারে। তবে এই ঝগড়ার নে...
...,...,...,...
2688,পাকা জামে রঙিন ঈশ্বরদীর মুলাডুলি পাইকারি বাজার,পল্লি কবি জসীম উদ্দীনের ‘মামার বাড়ী’ কবিতার ‘প...,পল্লি কবি জসীম উদ্দীনের ‘মামার বাড়ী’ কবিতার ‘প...
2689,কুমিল্লায় বিলুপ্তির পথে দেশীয় প্রজাতির মাছ,"কুমিল্লার খাল-বিল, পুকুর-জলাশয় থেকে দেশীয় বিভি...","কুমিল্লার খাল-বিল, পুকুর-জলাশয় থেকে দেশীয় বিভি..."
2690,রেমালের ধাক্কা কাটিয়ে উঠছে সুন্দরবন,দীর্ঘদিনের দাবদাহ ও ঘূর্ণিঝড় রেমালের উচ্চ জলোচ...,দীর্ঘদিনের দাবদাহ ও ঘূর্ণিঝড় রেমালের উচ্চ জলোচ...
2691,ফাইনালে দক্ষিণ আফ্রিকা ফেভারিট,‘ফাইনালে দক্ষিণ আফ্রিকা ফেভারিট’ শিরোনামটা পড়ে...,‘ফাইনালে দক্ষিণ আফ্রিকা ফেভারিট’ শিরোনামটা পড়ে...


Handle the missing values for exceptions

In [45]:
df = df[~df.apply(lambda x: '' in x.values, axis=1)] # Remove rows with any empty cells


In [46]:
# Uncomment if start & end dates are not defined. Set the date as per as your link dataframe/dataset
#start_date = date(2024,6,5) 
#end_date = date(2024,6,8)

In [47]:
# df.to_csv(f'News From 2023-06-01 To 2023-06-10.csv', index=False) # Save the data to a CSV file

In [48]:
df.to_csv(f'News From {start_date} To {end_date}.csv', index=False) # Save the data to a CSV file

In [49]:
df['headline'].iloc[3]

'রাজধানীতে মাদকবিরোধী অভিযানে গ্রেপ্তার ১৯ '

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2686 entries, 0 to 2692
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  2686 non-null   object
 1   summary   2686 non-null   object
 2   article   2686 non-null   object
dtypes: object(3)
memory usage: 83.9+ KB
